## View Frustum et Illumination

In [2]:
reset()

def produit_scalaire(U,V):
    
    return U.column(0).dot_product(V.column(0))

def produit_vectoriel(U,V):
      
    W = U.column(0)[0:3].cross_product(V.column(0)[0:3])
    
    return matrix([[W[0]],[W[1]],[W[2]],[0]])

def normal_au_plan(P):
    
    U = P.matrix_from_columns([1])-P.matrix_from_columns([0])
    V = P.matrix_from_columns([2])-P.matrix_from_columns([0])
    W = produit_vectoriel(U,V)  
    
    return W/norm(W)

def produit_cpte_cpte(A,B):
    
    return matrix([[A[0,0]*B[0,0]],[A[1,0]*B[1,0]],[A[2,0]*B[2,0]]])

In [3]:

def afficher_polygone(P,color):
    
    return polygon(tuple(P[0:3].transpose()),color=color)

def afficher_mallaige_polygonal(V,C,color):
    
    P = 0
    for c in C:
        sommets = []
        for n in c:
            if n > -1:
                sommets.append(V.column(n)[0:3])
        P = P + polygon(sommets,opacity=0.5, color=color)
    return P

In [4]:
t = var('t')

def courbe_bezier(PC):
    
    B = matrix([[-1,3,-3,1],[3,-6,3,0],[-3,3,0,0],[1,0,0,0]])    
    T = matrix([[t^3],[t^2],[t],[1]])
    
    return PC*B*T

def afficher_courbe_parametrique(C,t,a,b,color):
    
    return parametric_plot3d(C.column(0)[0:3], (t,a,b), color = color, thickness = 4) 

def afficher_courbe_parametrique2D(C,t,a,b,color):
    
    return parametric_plot(C.column(0)[0:2], (t,a,b), color = color, thickness = 4) 

In [5]:
def translation(d): 
    
    return matrix([[1,0,0,d[0,0]],[0,1,0,d[1,0]],[0,0,1,d[2,0]],[0,0,0,1]]) 

def rotation_axe(eje,angulo):
    
    if eje == 'x':
        M = matrix([[1,0,0,0],[0,cos(angulo),-sin(angulo),0],[0,sin(angulo),cos(angulo),0],[0,0,0,1]])
    elif eje == 'y':
        M = matrix([[cos(angulo),0,sin(angulo),0],[0,1,0,0],[-sin(angulo),0,cos(angulo),0],[0,0,0,1]])    
    else:
        M = matrix([[cos(angulo),-sin(angulo),0,0],[sin(angulo),cos(angulo),0,0],[0,0,1,0],[0,0,0,1]])   
        
    return M

def homothetie(r):  
         
    return matrix([[r[0,0],0,0,0],[0,r[1,0],0,0],[0,0,r[2,0],0],[0,0,0,1]]) 

def simetria(n):
    
    n = n/norm(n)
    I = matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
    
    return I-2*n*n.transpose()

def shear(d,ang1,ang2):
    
    if d == 'x':
        M = matrix([[1, tan(ang1), tan(ang2), 0],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
    
    elif d == 'y':
        M = matrix([[1 ,0 ,0 ,0],[ tan(ang1), 1, tan(ang2), 0],[0, 0, 1, 0],[0, 0, 0, 1]])
    
    else:
        M = matrix([[1, 0, 0, 0],[0, 1, 0, 0],[tan(ang1), tan(ang2), 1, 0],[0, 0, 0, 1]])
        
    return M

In [6]:
def transf_camera(e,c,up):
    at = c-e
    w = -at/norm(at)
    u = produit_vectoriel(up,w)
    u = u/norm(u)
    v = produit_vectoriel(w,u) 
    
    a = u[0,0]*e[0,0]+u[1,0]*e[1,0]+u[2,0]*e[2,0]
    b = v[0,0]*e[0,0]+v[1,0]*e[1,0]+v[2,0]*e[2,0]
    c = w[0,0]*e[0,0]+w[1,0]*e[1,0]+w[2,0]*e[2,0]

    M_cam = matrix([[u[0,0],u[1,0],u[2,0],-a],[v[0,0],v[1,0],v[2,0],-b],[w[0,0],w[1,0],w[2,0],-c],[0,0,0,1]])
    return M_cam

def transf_persp(near,far):
    M_persp = matrix([[near, 0, 0, 0],[0, near, 0, 0],[0, 0, near+far, -near*far],[0, 0, 1, 0]])
    return M_persp


def transf_morth(l,r,b,top,near,far):
    M_morth = matrix([[2/(r-l), 0, 0, -(r+l)/(r-l)],[0, 2/(top-b), 0, -(top+b)/(top-b)],[0, 0, 2/(near-far), -(near+far)/(near-far)],[0, 0, 0, 1]])
    return M_morth


def transf_vp(nx,ny):
    M_vp = matrix([[nx/2, 0, 0, (nx-1)/2],[0, ny/2, 0, (ny-1)/2],[0, 0, 1, 0],[0, 0, 0, 1]])
    return M_vp

def simplifyMatrix(matrix):
    Smatrix = matrix
    for j in range(matrix.ncols()):
        for i in range(matrix.nrows()):
            Smatrix[i,j] = matrix[i,j]/matrix[matrix.nrows()-1,j]
    return Smatrix   


def afficherFrustum( l,r,b,top,near,far ):
    
    #Vértices del frustum
    V = matrix([[r, r, l, l, (far/near)*r, (far/near)*r, (far/near)*l, (far/near)*l],[b, top, top, b, (far/near)*b, (far/near)*top, (far/near)*top, (far/near)*b],[near, near, near, near, far, far, far, far]])
    #Caras del frustum
    C = matrix([[0,1,2,3],[4,5,6,7],[1,2,6,5],[0,3,7,4],[2,6,7,3],[1,5,4,0]])
    
    return afficher_mallaige_polygonal(V,C,'black')

In [7]:
def distance(A,B):
    
    dist = sqrt((A[0,0]-B[0,0])^2+(A[1,0]-B[1,0])^2+(A[2,0]-B[2,0])^2)
    return dist.n()

def distance_en_pixels(A,B):
    
    dist = sqrt((A[0,0]-B[0,0])^2+(A[1,0]-B[1,0])^2)
    return dist.n()

# View frustum culling

def centre_sphere(V):
    return matrix( [[sum(V.row(0))/V.ncols()], [sum(V.row(1))/V.ncols()], [sum(V.row(2))/V.ncols()], [1]])

def rayon_sphere(centre,V):
    return max([distance(centre,V.matrix_from_columns([i])) for i in range(V.ncols())] )

def distance_centreSRC_viewFrustum(centreSRC,l,r,b,top,near,far):
    #Vecteurs normaux aux plans du viewfrustum
    nnear = matrix( [[0], [0], [1], [0]])
    nfar = matrix( [[0], [0], [-1], [0]])
    nright = matrix( [[-near], [0], [r], [0]])
    nleft = matrix( [[near], [0], [-l], [0]])
    ntop = matrix( [[0], [-near], [top], [0]])
    nbottom = matrix( [[0], [near], [-b], [0]])
    dnear = abs(centreSRC[2][0]-near)
    dfar = abs(centreSRC[2][0]-far)
    dright = abs(produit_scalaire(centreSRC,nright))/norm(nright)
    dleft = abs(produit_scalaire(centreSRC,nleft))/norm(nleft)
    dtop = abs(produit_scalaire(centreSRC,ntop))/norm(ntop)
    dbottom = abs(produit_scalaire(centreSRC,nbottom))/norm(nbottom)
    
    show('distance plan left=',dleft.n())
    show('distance plan right=',dright.n()) 
    show('distance plan bottom=',dbottom.n())
    show('distance plan top=',dtop.n())
    show('distance plan near=',dnear.n())
    show('distance plan far=',dfar.n())
    
    return dleft,dright,dbottom,dtop,dnear,dfar

#small culling

def bounding_box(Vscreen):    
    P = matrix([[min(Vscreen.row(0))],[min(Vscreen.row(1))],[1]])
    show('P=',P)
    Q = matrix([[max(Vscreen.row(0))],[max(Vscreen.row(1))],[1]])
    show('Q=',Q)
    return P,Q

#backface culling

def backface_culling(V,C,at):
    
    P = []
    for c in C:
        n = normal_au_plan(V.matrix_from_columns([c[0],c[1],c[2]]))
        p=produit_scalaire(n,at)
        P.append(p)
    return P

In [8]:


def intersect_cote_plan(A,B,l,r,b,top,near,far):
    
    nnear = matrix( [[0], [0], [1], [0]])
    nfar = matrix( [[0], [0], [-1], [0]])
    nright = matrix( [[-near], [0], [r], [0]])
    nleft = matrix( [[near], [0], [-l], [0]])
    ntop = matrix( [[0], [-near], [top], [0]])
    nbottom = matrix( [[0], [near], [-b], [0]])
    
    #Parametres
    Qnear = matrix( [[0], [0], [near], [1]])
    Qfar = matrix( [[0], [0], [far], [1]])
    
    if produit_scalaire(nleft,B-A)==0:
        tleft = 100000000
        show('Le cote est parallele au plan left')
    else:
        tleft = produit_scalaire(nleft,-A)/produit_scalaire(nleft,B-A)
           
    if produit_scalaire(nright,B-A)==0:
        tright = 100000000
        show('Le cote est parallele au plan right')
    else:
        tright = produit_scalaire(nright,-A)/produit_scalaire(nright,B-A)
        
    if produit_scalaire(nbottom,B-A)==0:
        tbottom = 100000000
        show('Le cote est parallele au plan bottom')
    else:
        tbottom = produit_scalaire(nbottom,-A)/produit_scalaire(nbottom,B-A)
        
    if produit_scalaire(ntop,B-A)==0:
        ttop = 100000000
        show('Le cote est parallele au plan top')
    else:
        ttop = produit_scalaire(ntop,-A)/produit_scalaire(ntop,B-A)
           
    if produit_scalaire(nnear,B-A)==0:
        tnear = 100000000
        show('Le cote est parallele au plan near')
    else:
        tnear = produit_scalaire(nnear,Qnear-A)/produit_scalaire(nnear,B-A)
        
    if produit_scalaire(nfar,B-A)==0:
        tfar = 100000000
        show('Le cote et le plan far sont paralleles')
    else:
        tfar = produit_scalaire(nfar,Qfar-A)/produit_scalaire(nfar,B-A)
    
    show('Parametre plan left= ',tleft,'=',tleft.n())
    show('Parametre plan right= ',tright,'=',tright.n()) 
    show('Parametre plan bottom= ',tbottom,'=',tbottom.n())
    show('Parametre plan top= ',ttop,'=',ttop.n())
    show('Parametre plan near= ',tnear,'=',tnear.n())
    show('Parametre plan far= ',tfar,'=',tfar.n())
    
    return tleft,tright,tbottom,ttop,tnear,tfar


In [9]:

def ilumination_Phong(F,e,V,n,IF,LA,kA,kD,kS,p):
    
    n = n/norm(n)
    show('n=',n)
    
    l = (F-V)/norm(F-V)
    show('l=',l)
    
    cosPhi = produit_scalaire(l,n)
    show('cosPhi=',cosPhi) 
        
    v = (e-V)/norm(e-V)
    show('v=',v)    
    
    r = 2*cosPhi*n-l
    show('r=',r)
    
    cosAlpha = produit_scalaire(r,v)
    show('cosAlpha=',cosAlpha)
    
    IA=produit_cpte_cpte(kA,LA)
    show('IA=',IA)
    
    ID=max(0,cosPhi)*produit_cpte_cpte(kD,IF)
    show('ID=',ID)
    
    IS=(max(0,cosAlpha))^p*produit_cpte_cpte(kS,IF)
    show('IS=',IS)
    
    IV = IA+ID+IS
    show('IV=', IV)
    
    return IV*255  

def coordonnees_baricentre(A,B,C,P):
    
    areaABC=det(matrix([[C[0,0]-A[0,0],B[0,0]-A[0,0]],[C[1,0]-A[1,0],B[1,0]-A[1,0]]]))    
    b=det(matrix([[C[0,0]-A[0,0],P[0,0]-A[0,0]],[C[1,0]-A[1,0],P[1,0]-A[1,0]]]))/areaABC
    c=det(matrix([[B[0,0]-A[0,0],P[0,0]-A[0,0]],[B[1,0]-A[1,0],P[1,0]-A[1,0]]]))/(-areaABC)
    a=1-b-c
    return a,b,c

def composante_profendeur(A,B,C,P):

    barP=coordonnees_baricentre(A,B,C,P)
    return barP[0]*A[2,0]+barP[1]*B[2,0]+barP[2]*C[2,0]

### Exemple:

In [10]:

F = matrix([[-5], [-5], [5], [1]])
e = matrix([[0], [0], [0], [1]])
V = matrix([[1], [-1], [-5], [1]])
n =  matrix([[-0.1], [-0.9], [-0.4], [0]])
IF = matrix([[1], [ 1], [ 1]])
LA = matrix([[0.2], [ 0.2 ], [0.2]])
kA = matrix([ [0.24725], [0.1995], [0.0745]])
kD = matrix([ [0.75164], [ 0.60648], [ 0.22648]])
kS = matrix([[0.628281], [ 0.555802], [ 0.36606]])
p = 1

In [11]:
ilumination_Phong(F,e,V,n,IF,LA,kA,kD,kS,p)

'n=' [-0.101015254455221]
[-0.909137290096990]
[-0.404061017820884]
[ 0.000000000000000]

'l=' [-0.48666426339228763]
[ -0.3244428422615251]
[  0.8111071056538127]
[                 0.0]

'cosPhi=' 0.016386838133611525

'v=' [-0.19245008972987526]
[ 0.19245008972987526]
[  0.9622504486493764]
[                 0.0]

'r=' [ 0.483353622144721]
[ 0.294647071033426]
[-0.824349670644079]
[ 0.000000000000000]

...

'IA=' [0.0494500000000000]
[0.0399000000000000]
[0.0149000000000000]

'ID=' [ 0.0123170030147478]
[0.00993828959127272]
[0.00371129110050034]

'IS=' [0.000000000000000]
[0.000000000000000]
[0.000000000000000]

'IV=' [0.0617670030147478]
[0.0498382895912727]
[0.0186112911005003]

[15.7505857687607]
[12.7087638457745]
[4.74587923062759]

### Ombres FLAT:

In [12]:

V = matrix([[1,-1,0,2,0],[-1,0,1,0,0],[0,0,0,0,2],[1,1,1,1,1]])
show('V=',V)
e = matrix([[0],[0],[5],[1]])
c = matrix([[0],[0],[2],[1]])
up = matrix([[0],[1],[0],[0]])
Mcam = transf_camera(e,c,up)
show('Mcam=',Mcam)
Vcam = Mcam*V
show('Vcam=',Vcam)

'V=' [ 1 -1  0  2  0]
[-1  0  1  0  0]
[ 0  0  0  0  2]
[ 1  1  1  1  1]

'Mcam=' [ 1.0  0.0  0.0 -0.0]
[ 0.0  1.0  0.0 -0.0]
[ 0.0  0.0  1.0 -5.0]
[ 0.0  0.0  0.0  1.0]

'Vcam=' [ 1.0 -1.0  0.0  2.0  0.0]
[-1.0  0.0  1.0  0.0  0.0]
[-5.0 -5.0 -5.0 -5.0 -3.0]
[ 1.0  1.0  1.0  1.0  1.0]

In [13]:

kA =matrix( [ [0.24725], [0.1995], [0.0745]])
kD = matrix( [ [0.75164], [ 0.60648], [ 0.22648]])
kS = matrix( [[0.628281], [ 0.555802], [ 0.36606]])
F = matrix( [[-5], [-5], [5], [1]])
e = matrix( [[0], [0], [0], [1]])
IF = matrix( [[1], [ 1], [ 1]])
LA = matrix( [[0.2], [ 0.2 ], [0.2]])
Vnew = matrix([[1,-1,0,1.3397,1.0981,1.3397,0],[-1,0,1,-0.6603,0,0.3301,0],[-5,-5,-5,-5,-4.0981,-5,-3],[1,1,1,1,1,1,1]])
show('Vnew=',Vnew)
V1 = Vnew.matrix_from_columns([0])
V7 = Vnew.matrix_from_columns([6])
V2 = Vnew.matrix_from_columns([1])
V = (V1+V7+V2)/3
show('V=',V)
n = produit_vectoriel(V7-V1,V2-V1)
n = n/norm(n)
#show('n=',n)
p = 1
ilumination_Phong(F,e,V,n,IF,LA,kA,kD,kS,p)

'Vnew=' [  1.00000000000000  -1.00000000000000  0.000000000000000   1.33970000000000   1.09810000000000   1.33970000000000  0.000000000000000]
[ -1.00000000000000  0.000000000000000   1.00000000000000 -0.660300000000000  0.000000000000000  0.330100000000000  0.000000000000000]
[ -5.00000000000000  -5.00000000000000  -5.00000000000000  -5.00000000000000  -4.09810000000000  -5.00000000000000  -3.00000000000000]
[  1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000]

'V=' [ 0.000000000000000]
[-0.333333333333333]
[ -4.33333333333333]
[  1.00000000000000]

'n=' [-0.436435780471985]
[-0.872871560943970]
[ 0.218217890235992]
[ 0.000000000000000]

'l=' [-0.432113401436758]
[-0.403305841340974]
[ 0.806611682681948]
[ 0.000000000000000]

'cosPhi=' 0.716641048512131

...

'IA=' [0.0494500000000000]
[0.0399000000000000]
[0.0149000000000000]

'ID=' [0.538656077703658]
[0.434628463101637]
[0.162304864667027]

'IS=' [0.000000000000000]
[0.000000000000000]
[0.000000000000000]

'IV=' [0.588106077703658]
[0.474528463101637]
[0.177204864667027]

[149.967049814433]
[121.004758090918]
[45.1872404900920]

### EXEMPLE GOURAUD:

In [14]:

kA =matrix( [ [0.24725], [0.1995], [0.0745]])
kD = matrix( [ [0.75164], [ 0.60648], [ 0.22648]])
kS = matrix( [[0.628281], [ 0.555802], [ 0.36606]])
F = matrix( [[-5], [-5], [5], [1]])
e = matrix( [[0], [0], [0], [1]])
IF = matrix( [[1], [ 1], [ 1]])
LA = matrix( [[0.2], [ 0.2 ], [0.2]])
Vnew = matrix([[1,-1,0,1.3397,1.0981,1.3397,0],[-1,0,1,-0.6603,0,0.3301,0],[-5,-5,-5,-5,-4.0981,-5,-3],[1,1,1,1,1,1,1]])
show('Vnew=',Vnew)
n1 = normal_au_plan(Vnew.matrix_from_columns([0,6,1]))+normal_au_plan(Vnew.matrix_from_columns([0,3,4]))+normal_au_plan(Vnew.matrix_from_columns([0,4,6]))
n1 = n1/norm(n1)
n7 = normal_au_plan(Vnew.matrix_from_columns([6,2,1]))+normal_au_plan(Vnew.matrix_from_columns([6,1,0]))+normal_au_plan(Vnew.matrix_from_columns([6,0,4]))+normal_au_plan(Vnew.matrix_from_columns([6,4,2]))
n7 = n7/norm(n7)
n2 = normal_au_plan(Vnew.matrix_from_columns([1,6,2]))+normal_au_plan(Vnew.matrix_from_columns([1,0,6]))
n2 = n2/norm(n2)
p=1
IV1 = ilumination_Phong(F,e,V1,n1,IF,LA,kA,kD,kS,p)
IV7 = ilumination_Phong(F,e,V7,n7,IF,LA,kA,kD,kS,p)
IV2 = ilumination_Phong(F,e,V2,n2,IF,LA,kA,kD,kS,p)

IB = 1/3*IV1+1/3*IV7+1/3*IV2

show('IV1=',IV1)
show('IV7=',IV7)
show('IV2=',IV2)
show('IB=',IB)

'Vnew=' [  1.00000000000000  -1.00000000000000  0.000000000000000   1.33970000000000   1.09810000000000   1.33970000000000  0.000000000000000]
[ -1.00000000000000  0.000000000000000   1.00000000000000 -0.660300000000000  0.000000000000000  0.330100000000000  0.000000000000000]
[ -5.00000000000000  -5.00000000000000  -5.00000000000000  -5.00000000000000  -4.09810000000000  -5.00000000000000  -3.00000000000000]
[  1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000]

'n=' [ 0.281469992041391]
[-0.794554788296511]
[ 0.538012390168951]
[ 0.000000000000000]

'l=' [-0.486664263392288]
[-0.324442842261525]
[ 0.811107105653813]
[ 0.000000000000000]

'cosPhi=' 0.557191900099395

'v=' [-0.192450089729875]
[ 0.192450089729875]
[ 0.962250448649376]
[ 0.000000000000000]

...

'IV=' [0.604217834121450]
[0.487528646743092]
[0.182059569836393]

'IV1=' [119.405718546631]
[96.3455646109317]
[35.9786694913003]

'IV7=' [287.433442657268]
[241.778865031496]
[122.262973290054]

'IV2=' [154.075547700970]
[124.319804919488]
[46.4251903082801]

'IB=' [186.971569634956]
[154.148078187305]
[68.2222776965447]

### EXEMPLE COORDS BARICENTRIQUES

In [15]:
A=matrix([[20],[30],[0.82],[1]])
B=matrix([[40],[60],[0.16],[1]])
C=matrix([[60],[30],[0.29],[1]])
P=matrix([[10],[10],[1]])
show(coordonnees_baricentre(A,B,C,P))
P=matrix([[40],[50],[1]])
show(coordonnees_baricentre(A,B,C,P))
composante_profendeur(A,B,C,P)
0.16*0.82+0.67*0.16+0.83*0.29

(1.58333333333333, -0.666666666666667, 0.0833333333333333)

(0.166666666666667, 0.666666666666667, 0.166666666666667)

0.479100000000000

### Exemple viewFrustum: 

In [16]:
V = matrix([[ 1, -1, 0, 2, 0],[-1,  0, 1, 0, 0],[ 0,  0, 0, 0, 2],[1,  1, 1, 1, 1]])   
show('V=',V)

C =matrix([[0, 1 ,2],[ 2,3,0],[0,3,4],[2,4,3],[1,4,2],[0,4,1]])

    
e = matrix([[0],[0],[5],[1]])
c = matrix([[0],[0],[2],[1]])
up = matrix([[0],[1],[0],[0]])

near = -1
far = -8

FOV = 100
nx=600
ny=400

'V=' [ 1 -1  0  2  0]
[-1  0  1  0  0]
[ 0  0  0  0  2]
[ 1  1  1  1  1]

In [17]:
r = abs(near)*tan((FOV*pi/180)/2)
l = -r
top = r
b = -r
Mcam = transf_camera(e,c,up)
show('Mcam=',Mcam)
Vcam = Mcam*V
show('Vcam=',Vcam)
Mpersp=transf_persp(near,far)
show('Mpersp=',Mpersp)
Morth=transf_morth(l,r,b,top,near,far)
show('Morth=',Morth.n())
Mvp=transf_vp(nx,ny)

'Mcam=' [ 1.0  0.0  0.0 -0.0]
[ 0.0  1.0  0.0 -0.0]
[ 0.0  0.0  1.0 -5.0]
[ 0.0  0.0  0.0  1.0]

'Vcam=' [ 1.0 -1.0  0.0  2.0  0.0]
[-1.0  0.0  1.0  0.0  0.0]
[-5.0 -5.0 -5.0 -5.0 -3.0]
[ 1.0  1.0  1.0  1.0  1.0]

'Mpersp=' [-1  0  0  0]
[ 0 -1  0  0]
[ 0  0 -9 -8]
[ 0  0  1  0]

'Morth=' [0.839099631177280 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.839099631177280 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.285714285714286  1.28571428571429]
[0.000000000000000 0.000000000000000 0.000000000000000  1.00000000000000]

#### Supposons que FOV=100 degres, montrons que l'objet et totalement contenu dans le view frustum.

In [18]:
# View frustum culling

centre=centre_sphere(V)
show('rayon de la sphere=',rayon_sphere(centre,V))

centreSRC=Mcam*centre
distance_centreSRC_viewFrustum(centreSRC,l,r,b,top,near,far)

show('coordonnees du centre de la sphere dans le colume canonique=',simplifyMatrix(Morth*Mpersp*centreSRC).n())
     
afficherFrustum( l,r,b,top,near,far )+afficher_mallaige_polygonal(Vcam,C,'red')

'rayon de la sphere=' 1.64924225024706

'distance plan left=' 3.78091948222192

'distance plan right=' 3.26668939447268

'distance plan bottom=' 3.52380443834730

'distance plan top=' 3.52380443834730

'distance plan near=' 3.60000000000000

'distance plan far=' 3.40000000000000

'coordonnees du centre de la sphere dans le colume canonique=' [0.0729651853197635]
[ 0.000000000000000]
[-0.788819875776398]
[  1.00000000000000]

Graphics3d Object

#### Suposons que la dimension de la fenetre est de 600 x 400 pixels, avec un seuil de 50 pixels, determinons si l'objet s'affichera ou non dans la fenetre.

In [19]:
#Small culling

Vpersp=Mpersp*Vcam
show('Vpersp=',Vpersp)
Vorth=Morth*Vpersp
show('Vorth=',Vorth.n())
Vvp=Mvp*Vorth
show('Vvp=',Vvp.n())
Vscreen=simplifyMatrix(Vvp.n())
show('Vscreen=',Vscreen)

bbox=bounding_box(Vscreen)
show('dimension de la bounding box=',distance_en_pixels(bbox[0],bbox[1]))

'Vpersp=' [-1.0  1.0  0.0 -2.0  0.0]
[ 1.0  0.0 -1.0  0.0  0.0]
[37.0 37.0 37.0 37.0 19.0]
[-5.0 -5.0 -5.0 -5.0 -3.0]

'Vorth=' [-0.839099631177280  0.839099631177280  0.000000000000000  -1.67819926235456  0.000000000000000]
[ 0.839099631177280  0.000000000000000 -0.839099631177280  0.000000000000000  0.000000000000000]
[  4.14285714285714   4.14285714285714   4.14285714285714   4.14285714285714   1.57142857142857]
[ -5.00000000000000  -5.00000000000000  -5.00000000000000  -5.00000000000000  -3.00000000000000]

'Vvp=' [-1749.22988935318 -1245.77011064682 -1497.50000000000 -2000.95977870637 -898.500000000000]
[-829.680073764544 -997.500000000000 -1165.31992623546 -997.500000000000 -598.500000000000]
[ 4.14285714285714  4.14285714285714  4.14285714285714  4.14285714285714  1.57142857142857]
[-5.00000000000000 -5.00000000000000 -5.00000000000000 -5.00000000000000 -3.00000000000000]

'Vscreen=' [  349.845977870637   249.154022129363   299.500000000000   400.191955741274   299.500000000000]
[  165.936014752909   199.500000000000   233.063985247091   199.500000000000   199.500000000000]
[-0.828571428571429 -0.828571428571429 -0.828571428571429 -0.828571428571429 -0.523809523809523]
[  1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000]

'P=' [249.154022129363]
[165.936014752909]
[1.00000000000000]

'Q=' [400.191955741274]
[233.063985247091]
[1.00000000000000]

'dimension de la bounding box=' 165.283458980092

#### Determinons si toutes les faces de l'objet sont visibles

In [20]:
#produit scalaire < 0 --> visible
at=c-e
show(at)
backface_culling(V,C,at)

[ 0]
[ 0]
[-3]
[ 0]

[3.0, 3.0, -1.7320508075688776, -1.2247448713915892, -1.0, -0.6546536707079772]

### Exemple:

In [21]:
V = matrix([[ 1, -1, 0, 2, 0],[-1,  0, 1, 0, 0],[ 0,  0, 0, 0, 2],[1,  1, 1, 1, 1]])   
show('V=',V)

C =matrix([[0, 1 ,2],[ 2,3,0],[0,3,4],[2,4,3],[1,4,2],[0,4,1]])

    
e = matrix([[0],[0],[5],[1]])
c = matrix([[0],[0],[2],[1]])
up = matrix([[0],[1],[0],[0]])

near = -1
far = -8

FOV = 30
nx=600
ny=400

'V=' [ 1 -1  0  2  0]
[-1  0  1  0  0]
[ 0  0  0  0  2]
[ 1  1  1  1  1]

In [22]:
r = abs(near)*tan((FOV*pi/180)/2)
l = -r
top = r
b = -r
Mcam = transf_camera(e,c,up)
show('Mcam=',Mcam)
Vcam = Mcam*V
show('Vcam=',Vcam)
Mpersp=transf_persp(near,far)
show('Mpersp=',Mpersp)
Morth=transf_morth(l,r,b,top,near,far)
show('Morth=',Morth.n())
Mvp=transf_vp(nx,ny)

'Mcam=' [ 1.0  0.0  0.0 -0.0]
[ 0.0  1.0  0.0 -0.0]
[ 0.0  0.0  1.0 -5.0]
[ 0.0  0.0  0.0  1.0]

'Vcam=' [ 1.0 -1.0  0.0  2.0  0.0]
[-1.0  0.0  1.0  0.0  0.0]
[-5.0 -5.0 -5.0 -5.0 -3.0]
[ 1.0  1.0  1.0  1.0  1.0]

'Mpersp=' [-1  0  0  0]
[ 0 -1  0  0]
[ 0  0 -9 -8]
[ 0  0  1  0]

'Morth=' [ 3.73205080756888 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000  3.73205080756888 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 0.285714285714286  1.28571428571429]
[0.000000000000000 0.000000000000000 0.000000000000000  1.00000000000000]

In [23]:
# View frustum culling

centre=centre_sphere(V)
show('rayon de la sphere=',rayon_sphere(centre,V))

centreSRC=Mcam*centre
distance_centreSRC_viewFrustum(centreSRC,l,r,b,top,near,far)

show('Coordonnees du centre de la sphere dans le volume canonique=',simplifyMatrix(Morth*Mpersp*centreSRC).n())
     
afficherFrustum( l,r,b,top,near,far )+afficher_mallaige_polygonal(Vcam,C,'green')

'rayon de la sphere=' 1.64924225024706

'distance plan left=' 1.57693793798722

'distance plan right=' 0.804197276955968

'distance plan bottom=' 1.19056760747160

'distance plan top=' 1.19056760747160

'distance plan near=' 3.60000000000000

'distance plan far=' 3.40000000000000

'Coordonnees du centre de la sphere dans le volume canonique=' [ 0.324526157179902]
[ 0.000000000000000]
[-0.788819875776398]
[  1.00000000000000]

Graphics3d Object

In [24]:
Vproy = simplifyMatrix(Morth*Mpersp*Mcam*V)
show(Vproy.n())

[ 0.746410161513775 -0.746410161513775  0.000000000000000   1.49282032302755  0.000000000000000]
[-0.746410161513775  0.000000000000000  0.746410161513775  0.000000000000000  0.000000000000000]
[-0.828571428571429 -0.828571428571429 -0.828571428571429 -0.828571428571429 -0.523809523809524]
[  1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000   1.00000000000000]

In [25]:
#V4 en dehors du view frustum. clipping V1V4v5 et V3V4V5


V1 = Vcam.matrix_from_columns([0])
V4 = Vcam.matrix_from_columns([3])
tplans=intersect_cote_plan(V1,V4,l,r,b,top,near,far)

'Le cote est parallele au plan near'

'Le cote et le plan far sont paralleles'

'Parametre plan left= ' 5.0*sqrt(3) - 11.0 '=' -2.33974596215561

'Parametre plan right= ' -5.0*sqrt(3) + 9.0 '=' 0.339745962155614

'Parametre plan bottom= ' 5.0*sqrt(3) - 9.0 '=' -0.339745962155614

'Parametre plan top= ' -5.0*sqrt(3) + 11.0 '=' 2.33974596215561

'Parametre plan near= ' 100000000 '=' 1.00000000000000e8

'Parametre plan far= ' 100000000 '=' 1.00000000000000e8

In [26]:
#coupe le plan droit
#Point d'intersection
Q1 = V1 + tplans[1]*(V4-V1)
show('Q1=',Q1.n())
#...

'Q1=' [  1.33974596215561]
[-0.660254037844386]
[ -5.00000000000000]
[  1.00000000000000]